# Lab 3: Document Classification (Part 2)

### Overview
This topic builds on the activities of the previous topic on sentiment analysis. You will be focussing on the movie review corpus with a view to investigating the following issues.

- Evaluation metrics for classifier performance
- What is the impact of varying training data size? To what extent does increasing the quantity of training data improve classifier performance?

By this stage, you should be very comfortable with Python's [list comprehensions](http://docs.python.org/tutorial/datastructures.html#list-comprehensions) and [slice](http://bergbom.blogspot.co.uk/2011/04/python-slice-notation.html) notation.



>To access functionality defined in previous notebooks, copy the functions defined in Week3Labs into a `utils.py` file and then import it into the notebook.  There is a `utils.py` file included with these resources which you can update.

In [1]:
##uncomment the following lines on colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#other needed imports
from nltk.corpus import movie_reviews
import random
import pandas as pd
#this next line will ensure pandas graphs are plotted in the notebook
%matplotlib inline

##download stopwords and movie_reviews - need to do this every session on colab
import nltk
nltk.download('stopwords')
nltk.download('movie_reviews')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [ ]:
#set your system path so that jupyter knows where to look for other files
import sys
sys.path.append('/content/drive/My Drive/NLE Notebooks 2021/Week3LabsSolutions/')
#import code to setup training and testing data, wordlist classifiers and NB classifiers
from utils import *



In [6]:
import random # have a look at the documentation at https://docs.python.org/3/library/random.html


def split_data(data, ratio=0.7): # when the second argument is not given, it defaults to 0.7
    """
    Given collection of items and ratio:
     - partitions the collection into training and testing, where the proportion in training is ratio,

    :param data: A list (or generator) of documents or doc ids
    :param ratio: The proportion of training documents (default 0.7)
    :return: a pair (tuple) of lists where the first element of the
            pair is a list of the training data and the second is a list of the test data.
    """

    n = len(data)  #Found out number of samples present.  data could be a list or a generator
    train_indices = random.sample(range(n), int(n * ratio))          #Randomly select training indices
    test_indices = list(set(range(n)) - set(train_indices))   #Other items are testing indices

    train = [data[i] for i in train_indices]           #Use training indices to select data
    test = [data[i] for i in test_indices]             #Use testing indices to select data

    return (train, test)                       #Return split data

In [14]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def normalized_tokens(data):

  # C normalisation
  case_normalised_token = [token.lower() for token in data]

  # Replace digits with "NUM"
  normalised_token_first = ["NUM" if token.isdigit() else token for token in case_normalised_token]

  # Replace ordinals with "Nth"
  normalised_token = ["Nth" if (token.endswith(("nd","st","th","rd")) and token[:-2].isdigit()) else token for token in normalised_token_first]

  return normalised_token

def stopword_removal(data):
  stop = stopwords.words('english')

  # Remove stopwords and token not made of only letters
  filtered_token_first = [token for token in data if token.isalpha() and token not in stop ]

  # Remove "NUM" and "Nth"
  filtered_token = [token for token in filtered_token_first if token not in ["NUM", "Nth"]]

  return filtered_token

def lemmatize_tokens(data):
  lem = WordNetLemmatizer()

  # Lemmatize the tokens
  lemmatize_token = [lem.lemmatize(token) for token in data]

  return lemmatize_token

# Perform full preprocessing
def normalise(data):
  normalised_reviews = normalized_tokens(data)
  filtered_reviews = stopword_removal(normalised_reviews)
  cleaned_data = lemmatize_tokens(filtered_reviews)

  return cleaned_data

In [24]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [25]:
from nltk.probability import FreqDist

## Evaluation Metrics for Classifier Performance

### Accuracy
Here is code for an evaluation function <code>evaluate_wordlist_classifier</code> which can be used to determine how well a word_list classifier performs. This function returns the <b>accuracy</b> of a classifier. The accuracy metric is defined as the proportion of documents that were correctly classified.  Look at the code and make sure you understand what it is doing

In [26]:
def classifier_evaluate(cls, test_data):
    '''
    cls: an instance of a classifier object which has a classify method which returns "pos" or "neg"
    test_data: a list of pairs where each pair is a FreqDist rep of a doc and its label

    returns: float point number which is the accuracy of the classifier on the test data provided
    '''
    acc = 0
    docs,goldstandard=zip(*test_data) #note this neat pythonic way of turning a list of pairs into a pair of lists
    #pass all of the docs to the classifier and get back a list of predictions
    predictions=cls.classify_many(docs)
    #zip the predictions with the goldstandard labels and compare
    for prediction,goldlabel in zip(predictions,goldstandard):
        if prediction==goldlabel:
            acc+=1

    return acc / (len(test_data))

Now we need some data to train and test our classifier on.  We are going to make use of the code from the lab_3_1.  

In [27]:
def get_train_test_data():

    #get ids of positive and negative movie reviews
    pos_review_ids=movie_reviews.fileids('pos')
    neg_review_ids=movie_reviews.fileids('neg')

    #split positive and negative data into training and testing sets
    pos_train_ids, pos_test_ids = split_data(pos_review_ids)
    neg_train_ids, neg_test_ids = split_data(neg_review_ids)
    #add labels to the data and concatenate
    training = [(movie_reviews.words(f),'pos') for f in pos_train_ids]+[(movie_reviews.words(f),'neg') for f in neg_train_ids]
    testing = [(movie_reviews.words(f),'pos') for f in pos_test_ids]+[(movie_reviews.words(f),'neg') for f in neg_test_ids]
    #now normalise and create bag-of-words FreqDist representations
    training_norm=[(FreqDist(normalise(wordlist)),label) for (wordlist,label) in training]
    testing_norm=[(FreqDist(normalise(wordlist)),label) for (wordlist,label) in testing]
    return training_norm, testing_norm

In [28]:
random.seed(41)  #set the random seeds so these random splits are always the same
training,testing=get_train_test_data()

training[0]

(FreqDist({'melvin': 6, 'simon': 4, 'dog': 4, 'day': 3, 'playing': 3, 'one': 2, 'serve': 2, 'carol': 2, 'threatens': 2, 'shut': 2, ...}),
 'pos')

We are now going to use this function to start evaluating our classifiers from last week.  Lets first try the SimpleClassifier

In [31]:
def most_frequent_words(freq1,freq2,k):

    difference = freq1-freq2  # frequency differences
    sorted_diff = difference.most_common()  # sorted by highest difference
    mostwords = [token for (token, freq) in sorted_diff[:k]]  # collect top k tokens

    return mostwords

In [34]:
from nltk.classify.api import ClassifierI

class SimpleClassifier(ClassifierI):

    def __init__(self, pos, neg):
        self._pos = pos
        self._neg = neg

    def classify(self, words):
        score = 0

        # add code here that assigns an appropriate value to score
        return "neg" if score < 0 else "pos"

    def labels(self):
        return ("pos", "neg")

In [35]:
#here I am going to create an instance of a very simple classifier

my_positive_word_list = ["good","great","lovely", "excellent"] # extend this one or put your own list here
my_negative_word_list = ["bad", "terrible", "awful", "dreadful"] # extend this one or put your own list here
movie_classifier1 = SimpleClassifier(my_positive_word_list,my_negative_word_list)

#Evaluate classifier
#The function requires two arguments:
# 1. Word list based classifer
# 2. A list (or generator) of labelled test items
score = classifier_evaluate(movie_classifier1, testing)
print(score)

0.5


If you have run the cell above without updating the SimpleClassifier code you should see that the accuracy is 0.5 i.e., 50%. The original SimpleClassifier just assigns everything to the positive class.  Since it is a binary classification decision and the classes are balanced, it will get 50% of the decisions correct (those that are positive) and 50% of the decisions incorrect (those that are actually negative).  This is the **baseline** result for this kind of classification task.  We obviously want to build classifiers that do better than this.

### Exercise 1

Now you try one of the classifiers that you wrote that selects positive and negative words from the training data.  Hopefully, this will perform better than the baseline of 50%.

In [ ]:
#Create a new classifier
#Make sure you have updated the code in utils.py to contain your WordList Classifier
#If you update the utils.py code mid-session, you will need to restart the runtime / kernel in order to force it to import the new updated code


In [36]:
def most_frequent_words(freq1,freq2,k):

    difference = freq1-freq2  # frequency differences
    sorted_diff = difference.most_common()  # sorted by highest difference
    mostwords = [token for (token, freq) in sorted_diff[:k]]  # collect top k tokens

    return mostwords

In [38]:
class SimpleClassifier(ClassifierI):

    def __init__(self, k):
        # Store k
        self._k = k

    def classify(self, doc):
        # Compute how many positive and negative tokens appear
        score = 0

        for word, count in doc.items():
          if word in self._pos:
            score += count  # Increase score for positive tokens
          if word in self._neg:
            score-= count  # Decrease score for negative tokens

        # Decide final label based on score
        if score > 0:
          return "pos"
        elif score == 0:
          return random.choice(["pos", "neg"])  # Resolve tied cases
        else:
          return "neg"

    def train(self,training_data):

        # Build frequency distributions for positive and negative reviews
        pos_freq_dist=FreqDist()
        neg_freq_dist=FreqDist()

        for reviewDist,label in training_data:
            if label=='pos':
                pos_freq_dist+=reviewDist
            else:
                neg_freq_dist+=reviewDist

        # Get the k most frequent tokens
        self._pos=most_frequent_words(pos_freq_dist,neg_freq_dist,self._k)
        self._neg=most_frequent_words(neg_freq_dist,pos_freq_dist,self._k)

    def labels(self):
        # Return the class labels
        return ("pos", "neg")

In [39]:
movie_classifier2 = SimpleClassifier(100)
movie_classifier2.train(training)

In [40]:
score = classifier_evaluate(movie_classifier2, testing)
print(score)

0.6483333333333333


## Precision, Recall and F1 score etc

When classes are unbalanced, evaluating classifiers in terms of accuracy can be misleading.  For example, if 10% of documents are relevant and 90% of documents are irrelevant, then a classifier which labels all documents as irrelevant will obtain an accuracy of 90%.  This sounds good but is actually useless. More useful metrics for evaluation of performance are precision, recall and F1 score.  These metrics allow us to distinguish the different types of errors our classifiers make.

For each class, $c$, we need to keep a record of
* True Positives: $TP=|\{i|\mbox{prediction}(i)=\mbox{label}(i)=c\}|$
* False Negatives: $FN=|\{i|\mbox{prediction}(i)\neq \mbox{label}(i)=c\}|$
* False Positives: $FP=|\{i|\mbox{label}(i) \neq \mbox{prediction}(i)=c\}|$
* True Negatives: $TN=|\{i|\mbox{prediction}(i)=\mbox{label}(i)\neq c\}|$

Note the symmetry in the binary classification task (the TN for one class are the TP for the other class and so on).  Therefore, in binary classification, we just record these values and compute the following evaluation metrics for a single class (e.g. "Relevant" or "Positive")

* Precision:
\begin{eqnarray*}
P=\frac{TP}{TP+FP}
\end{eqnarray*}
* Recall:
\begin{eqnarray*}
R=\frac{TP}{TP+FN}
\end{eqnarray*}
* F1-score:
\begin{eqnarray*}
F1 = \frac{2\times P\times R}{P+R}
\end{eqnarray*}


 ### Exercise 2.1

 The code below defines a ConfusionMatrix class for the binary classification task.  Currently, it will compute the number of TPs, FPs, FNs and TNs.  Test it out with predictions and test data for our sentiment analysis task.

In [ ]:
class ConfusionMatrix:
    def __init__(self,predictions,goldstandard,classes=("pos","neg")):

        (self.c1,self.c2)=classes
        self.TP=0
        self.FP=0
        self.FN=0
        self.TN=0
        for p,g in zip(predictions,goldstandard):
            if g==self.c1:
                if p==self.c1:
                    self.TP+=1
                else:
                    self.FN+=1

            elif p==self.c1:
                self.FP+=1
            else:
                self.TN+=1


    def precision(self):
        p=0
        #put your code to compute precision here

        return p

    def recall(self):
        r=0
        #put your code to compute recall here

        return r

    def f1(self):
        f1=0
        #put your code to compute f1 here

        return f1

### Exercise 2.2
* Add functionality to the ConfusionMatrix class code to compute precision, recall and F1 score
* Use your code to evaluate the performance of the different classifiers you have constructed.
* Interpret your results

## Investigating the impact of the quantity of training data
We will begin by exploring the impact on classification accuracy of using different quantities of training data.




Run the code in the cell below several times.  Each time it should generate a new sample of review data, train the classifiers and evaluate them.

In [ ]:

training,testing=get_train_test_data()

word_list_size = 100
threshold=100
classifiers={"Word List MF":SimpleClassifier_mf(word_list_size),
             "Word List Thresh":SimpleClassifier_ot(threshold)}

results=[]
for name,classifier in classifiers.items():
    classifier.train(training)
    accuracy=classifier_evaluate(classifier,testing)
    print("The accuracy of {} classifier is {}".format(name,accuracy))
    results.append((name,accuracy))

df = pd.DataFrame(results)
display(df)
ax = df.plot.bar(title="Experimental Results",legend=False,x=0)
ax.set_ylabel("Classifier Accuracy")
ax.set_xlabel("Classifier")
ax.set_ylim(0,1.0)

As you can see, the classifiers have different accuracies on different runs.

### Exercise 3.1
Copy the cell above and move the copy to be positioned below this cell. Then adapt the code so that the accuracy reported for each classifier is the average across multiple runs.

### Exercise 3.2
Adapt the code so that it calculates average precision, recall and F1-score rather than average accuracy.

### Exercise 3.3
Investigate the impact of training data size on the performance of both word list classifiers, using a range of subsets of movie reviews.

Hints and tips.
- You can copy the code cell that you created for the last exercise, and place the copy below this cell. Then adapt the code to determine accuracy, precision, recall and F1-score for each classifier on each subset.

- Use the `sample` function from the random module.
- Remember, the full data set has 1000 positive and 1000 negative reviews.
- You should continue to use 30% of the data for testing, so this means that we have up to 700 positive and 700 negative reviews to sample from.
- Make sure that you are selecting samples that have an equal number of positive and negative reviews.
- Consider (at least) the following sample sizes: 2, 10, 50, 100, 200, 400, 600 and 700.
- Note that the sample size is not the total number of reviews, but the number of positive reviews (which is also equal to the number of negative reviews).


**Interpret your results**

My answer:

